In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csvfrom pathlib import Path

In [ ]:
def parse_auto_ria(page=1):
    url = f"https://auto.ria.com/car/used/?page={page}"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
    }
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    data = []
    for ticket in soup.select("section.ticket-item"):
        item = {"Цена(в $)": "Na", "Пробег": "Na", "Модель_полная": "Na"}
        text = ticket.get_text(separator=" ", strip=True)

        # Цена
        price_tag = ticket.select_one("span.price-ticket") or ticket.select_one("span.green")
        if price_tag:
            item["Цена(в $)"] = price_tag.get_text(strip=True)

        # Пробег
        mileage = None
        for li in ticket.select("li"):
            li_text = li.get_text(strip=True)
            if re.search(r"[\d\s,]+км", li_text):
                mileage = re.sub(r"\s+", "", li_text).replace(",", "")
                break
        if mileage:
            item["Пробег"] = re.sub(r"[^\d]", "", mileage) + " тыс.км"

        # Модель (полная строка)
        model_tag = ticket.select_one("a.address")
        if model_tag:
            item["Модель_полная"] = model_tag.get_text(strip=True)

        data.append(item)
    return data

# Параметры: сколько страниц брать
NUM_PAGES = 30

# Сбор данных
all_data = []
for page in range(1, NUM_PAGES + 1):
    try:
        all_data += parse_auto_ria(page)
    except Exception as e:
        print(f"Ошибка на странице {page}: {e}")

# Формируем DataFrame и отброс «пустых» строк
df = pd.DataFrame(all_data)
df = df[(df != "Na").any(axis=1)].reset_index(drop=True)

# Разделяем «Модель_полная» на Марку и Модель
df[["Марка", "Модель"]] = df["Модель_полная"].str.extract(r"^(\S+)\s+(.+)$")

# Извлекаем год из «Модель» (если есть) и убираем его
df["Год"] = df["Модель"].str.extract(r"(19\d{2}|20\d{2})$")
df["Модель"] = df["Модель"].str.replace(r"(19\d{2}|20\d{2})$", "", regex=True).str.strip()

# Убираем временную колонку
df.drop(columns=["Модель_полная"], inplace=True)

# Вывод полной таблицы
from IPython.display import display
print("Все данные:")
display(df)

# Таблица только с записями, где нет ни одного Na
df_no_na = df[(df != "Na").all(axis=1)].reset_index(drop=True)
print("Отфильтрованные данные:")
display(df_no_na)

Все данные:


,Цена(в $),Пробег,Марка,Модель,Год
0,19 999,107 тыс.км,Audi,A3,2016
1,39 900,100 тыс.км,BMW,5 Series,2017
2,12 450,285 тыс.км,Mercedes-Benz,E-Class,2009
3,116 000,17 тыс.км,Audi,SQ8,2024
4,49 400,147 тыс.км,Mercedes-Benz,GLE-Class,2019
...,...,...,...,...,...
575,89 500,107 тыс.км,Porsche,Cayenne,2021
576,63 000,30 тыс.км,BMW,X5,2021
577,49 500,71 тыс.км,BMW,5 Series,2018
578,28 950,66 тыс.км,Volkswagen,Passat,2020


Отфильтрованные данные:


,Цена(в $),Пробег,Марка,Модель,Год
0,19 999,107 тыс.км,Audi,A3,2016
1,39 900,100 тыс.км,BMW,5 Series,2017
2,12 450,285 тыс.км,Mercedes-Benz,E-Class,2009
3,116 000,17 тыс.км,Audi,SQ8,2024
4,49 400,147 тыс.км,Mercedes-Benz,GLE-Class,2019
...,...,...,...,...,...
519,89 500,107 тыс.км,Porsche,Cayenne,2021
520,63 000,30 тыс.км,BMW,X5,2021
521,49 500,71 тыс.км,BMW,5 Series,2018
522,28 950,66 тыс.км,Volkswagen,Passat,2020


In [ ]:
# Сохранение полученных данных в data
data_dir = Path('../data').resolve()raw_path = data_dir / 'data_raw.csv'clean_path = data_dir / 'data_clean.csv'df.to_csv(raw_path, index=False)df_no_na.to_csv(clean_path, index=False)